In [1]:
import random
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import calendar
import logging
import time
from memory_profiler import memory_usage
import warnings

warnings.filterwarnings('ignore', message='No such comm target registered')

logging.basicConfig(filename='app.log', level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

#Generating Customer Data
def generate_customer():
    logger.info(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}-Generating customer...")
    #Customer_ID, age, gender
    customer_id = random.randint(1000000, 9999999)
    age = random.randint(20, 80)
    gender = random.choice(['Male', 'Female'])
    age_categories = {
        (20, 30): [0.75, 0.25],
        (30, 60): [0.25, 0.75],
        (60, 80): [0.5, 0.5]
    }
    #Marital Status based on age
    for age_range, marital_probs in age_categories.items():
        if age_range[0] <= age <= age_range[1]:
            marital_status = random.choices(['Single', 'Married'], weights=marital_probs)[0]
            break

    children_probs = {
        (20, 40): [0.4, 0.3, 0.2, 0.1, 0],
        (40, 80): [0.1, 0.3, 0.3, 0.2, 0.1]
    }
    
    #generating number of children based on age and children probability
    num_children = random.choices(range(5), weights=children_probs[(20, 40)] if age <= 40 else children_probs[(40, 80)])[0]

    education_probs = {
        (20, 25): [0.1, 0.5, 0.3, 0.1, 0],
        (25, 35): [0.1, 0.5, 0.3, 0.05, 0.05],
        (35, 80): [0.1, 0.5, 0.25, 0.1, 0.05]
    }

    #Education level numeric mapping
    for age_range, education_probs_list in education_probs.items():
        if age_range[0] <= age <= age_range[1]:
            education_level_mapping = {
                'No Education': 0,
                'High School': 1,
                "Bachelor's Degree": 2,
                "Master's Degree": 3,
                'Ph.D.': 4
            }
            education_level = random.choices(['No Education', 'High School', "Bachelor's Degree", "Master's Degree", 'Ph.D.'],
                           weights=education_probs_list)[0]
            education_level_numeric = education_level_mapping[education_level]
            break

    #Calculate Annual Income
    annual_income = 40 * 52 * ((15 + education_level_numeric * 10) + (age / 10) * 2)

    #Define marital status numeric value
    marital_status_numeric = 1 if marital_status == 'Married' else 0

    #Generating Number of accounts 
    num_accounts = marital_status_numeric + num_children + 1

    #For each customer, his acccounts are defined
    accounts = []

    #Calculate total credit line
    total_credit_line = num_accounts * (annual_income / 10)

    #Generate account info for each of the customer's account
    for i in range(num_accounts):
        account = generate_account_info(customer_id, i + 1, age, total_credit_line,num_accounts)
        accounts.append(account)
        total_credit_line += account['Account_Credit_Line']

    logger.info("Customer generation completed.")
    return {
        'Customer_ID': customer_id,
        'Age': age,
        'Gender': gender,
        'Marital_Status': marital_status,
        'Number_of_Children': num_children,
        'Education_Level': education_level,
        'Annual_Income': annual_income,
        'Number_of_Accounts': num_accounts,
        'Total_Credit_Line': total_credit_line,
        'Accounts': accounts
    }

#Generating Account data for each customer
def generate_account_info(customer_id, account_number, age, total_credit_line,num_accounts):
    logger.info(
        f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}-Generating account info for Customer ID: {customer_id}, Account Number: {account_number}...")
    max_date = min(datetime.now() - timedelta(days=365 * (age - 20)), datetime(2022, 1, 1))
    date_opened = datetime(random.randint(max_date.year, 2022), random.randint(1, 12), random.randint(1, 28))
    #Date opened should be before Jan 2022 and the customer should be 20 years old before he can open any account...

    #Generate AccountID by adding a number to the end of customer_id
    account_id = int(str(customer_id) + str(account_number))

    #Generate Account Credit Line, Annual Fee, Annual Interest Rate
    account_credit_line = random.uniform(0.1, 1.0) * (total_credit_line)

    annual_fee = account_credit_line * 0.01
    annual_interest_rate = random.uniform(0.15, 0.3)
    #credit line for each account of the customer
    num_accounts = 0
    total_credit_line_of_accounts = 0
    monthly_details = []

    logger.info("Account info generation completed.")
    return {
        'Account_ID': account_id,
        'Date_Opened': date_opened,
        'Account_Credit_Line': account_credit_line,
        'Annual_Fee': annual_fee,
        'Annual_Interest_Rate': annual_interest_rate,
        'Number_of_Accounts': num_accounts,
        'Total_Credit_Line_of_Accounts': total_credit_line_of_accounts,
        'Account_Status': 'open',
        'Monthly_Details': monthly_details
    }

#simulate account activity for each account and store monthly transactions data into account_activity_data.csv file
def simulate_account_activity(account):
    logger.info(
        f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}-Simulating account activity for Account ID: {account['Account_ID']}...")
    total_available_credit = account['Account_Credit_Line']  # Starting balance
    daily_interest_rate = account['Annual_Interest_Rate'] / 365
    credit_line = total_available_credit

    total_purchases_month = 0
    total_cash_advances_month = 0
    total_payments_month = 0
    total_interest_charged_month = 0
    current_balance = 0
    purchase_amount = 0

    # Delinquency variables
    past_due_amount = 0
    late_payment_fee = 30
    delinquency_counter = 0
    paymentMissed = False

    for month in range(1, 13):
        closing_balance = current_balance
        total_purchases_month = 0
        total_cash_advances_month = 0
        total_payments_month = 0
        total_interest_charged_month = 0
        current_balance = 0
        purchase_amount = 0
        total_payments_month = 0
        interest_charged = 0
        transaction_values = []
        transaction_Purchase_Counter = 0
        transaction_Cashout_Counter = 0
        total_transaction_amount = 0
        d = 0
        k = 0
        purchases_monthly = []
        cash_advances_monthly = []
        payments_monthly = []
        #purchases_monthly, cash_advances_monthly and payments_monthly are arrays containing the individual transaction amounts made
        num_days_in_month = calendar.monthrange(2022, month)[1]

        for day in range(1, num_days_in_month):
            # Purchase or Cash Advance
            d = random.randint(1, 7)   #d is transaction made in random day from 1 to 7
            #k is an integer value, k is added the value of d. such that it limits the days of transactions stays under total days in month
            k = k + d
            if k <= num_days_in_month and delinquency_counter < 3:
                chosen_event = random.choices(['Make a Purchase', 'Take Cash Out'], weights=[0.95, 0.05], k=1)[0]
                if chosen_event == 'Make a Purchase' and total_available_credit > 0 and delinquency_counter < 3:
                    # Make a purchase
                    purchase_amount = random.uniform(0, total_available_credit)
                    total_available_credit = credit_line - purchase_amount
                    current_balance += purchase_amount
                    interest_charged = interest_charged + (current_balance * daily_interest_rate * (day - 1))
                    total_purchases_month += purchase_amount
                    transaction_Purchase_Counter = transaction_Purchase_Counter + 1
                    purchases_monthly.append(purchase_amount)

                elif chosen_event == 'Take Cash Out' and total_available_credit > 0 and delinquency_counter < 3 and k <= num_days_in_month:
                    # Take cash advance
                    available_cash = min(total_available_credit, 0.1 * account['Account_Credit_Line'])
                    cash_advance_amount = random.uniform(0, available_cash)
                    current_balance += cash_advance_amount
                    interest_charged = interest_charged + (current_balance * daily_interest_rate * (day - 1))
                    total_cash_advances_month += cash_advance_amount
                    total_available_credit = credit_line - cash_advance_amount
                    transaction_Cashout_Counter = transaction_Cashout_Counter + 1
                    cash_advances_monthly.append(cash_advance_amount)

            # Apply daily interest
            current_balance *= (1 + daily_interest_rate)

            # Payment scenarios
            if random.randint(1, 10) == 1 and k < num_days_in_month and delinquency_counter < 3:
                individual_payments_record = current_balance
                total_payments_month = total_payments_month + current_balance
                payments_monthly.append(individual_payments_record)
                # 10% pay the exact amount they spent every day
                current_balance = 0
                paymentMissed = False
            elif random.randint(1, 7) == 1 and k < num_days_in_month and delinquency_counter < 3:
                individual_payments_record = current_balance
                total_payments_month = total_payments_month + current_balance
                payments_monthly.append(individual_payments_record)
                # 30% pay entire balance every d days
                current_balance = 0
                paymentMissed = False
            elif random.randint(1, 7) == 1 and k < num_days_in_month and delinquency_counter < 3:
                # 30% pay a proportion every d days
                days_to_pay = random.randint(1, 7)
                if day % days_to_pay == 0:
                    payment_proportion = random.uniform(0, 1)
                    payments_monthly.append(payment_proportion * current_balance)
                    current_balance *= (1 - payment_proportion)
                    paymentMissed = False
            elif delinquency_counter < 3 and k < num_days_in_month:
                # 30% follow payment period scenarios
                payment_period = random.randint(1, 10)
                if day <= payment_period and k < 32:
                    if random.random() < 0.1:
                        # 10% pay entire balance within the Payment Period
                        individual_payments_record = current_balance
                        total_payments_month += current_balance
                        payments_monthly.append(individual_payments_record)
                        current_balance = 0
                        paymentMissed = False
                    elif random.random() < 0.15:
                        # 15% pay only the Minimum Amount Due within the Payment Period
                        minimum_amount_due = 0.1 * current_balance
                        total_payments_month += minimum_amount_due
                        current_balance -= minimum_amount_due
                        payments_monthly.append(minimum_amount_due)
                        paymentMissed = False
                    else:
                        # miss payment
                        paymentMissed = True

        if day > 10 and paymentMissed == True and delinquency_counter < 3:
            past_due_amount = 0.1 * current_balance
            current_balance += past_due_amount + late_payment_fee
            delinquency_counter += 1

        # Calculate end-of-month metrics
        closing_balance = current_balance
        minimum_amount_due = 0.1 * closing_balance
        total_interest_charged_month += interest_charged
        total_transaction_amount = total_purchases_month + total_cash_advances_month

        account.setdefault('Monthly_Details', []).append({
            'Month': f"{calendar.month_name[month]} 2022",
            'Closing_Balance': closing_balance,
            'Minimum_Amount_Due': minimum_amount_due,
            'Total_Purchases': total_purchases_month,
            'Total_Cash_Advances': total_cash_advances_month,
            'Total_Payments': total_payments_month,
            'Total_Interest_Charged': total_interest_charged_month,
            'Total_Transaction_amount_in_month': total_transaction_amount,
            'Delinquency_Count': delinquency_counter,
            'Purchase_Transactions_of_month': transaction_Purchase_Counter,
            'Cash_advance_Transactions_of_month': transaction_Cashout_Counter,
            'Total_Transactions_of_month': transaction_Purchase_Counter + transaction_Cashout_Counter,
            'Total_Individual_Payments_of_month': payments_monthly,
            'Total_Individual_purchases_of_month': purchases_monthly,
            'Total_Individual_Cash_Advances_of_month': cash_advances_monthly
        })

        if delinquency_counter >= 3:
            logger.info(
                f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}-Account closed due to 3 consecutive missed payments. Account ID: {account['Account_ID']}")
            break

        # Reset monthly metrics
        total_purchases_month = 0
        total_cash_advances_month = 0
        total_payments_month = 0
        total_interest_charged_month = 0
        # print(delinquency_counter,'delinquency')

    logger.info(
        f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}-Account activity simulation completed for Account ID: {account['Account_ID']}.")


if __name__ == "__main__":
    start_time = time.time()

    # Generate 20,000 customers
    customers = [generate_customer() for _ in range(20000)]

    # Simulate account activity for each customer's accounts
    for customer in customers:
        for account in customer['Accounts']:
            simulate_account_activity(account)

    # Convert the data to dataframes
    customers_df = pd.DataFrame(customers)
    customers_df.drop('Accounts', axis=1, inplace=True)
    customers_df.to_csv('customers_data.csv', index=False)

    accounts_data = []
    for customer in customers:
        for account in customer['Accounts']:
            account_status = account.get('Account_Status', 'Open')  # 'Account_Status' default set to 'Open'
            accounts_data.append({
                'Customer_ID': customer['Customer_ID'],
                **account  # Unpack account details
            })

    accounts_df = pd.DataFrame(accounts_data)
    accounts_df.drop(['Number_of_Accounts', 'Total_Credit_Line_of_Accounts', 'Monthly_Details'], axis=1, inplace=True)
    accounts_df.to_csv('accounts_data.csv', index=False)

    account_activity_data = []
    for customer in customers:
        for account in customer['Accounts']:
            for monthly_detail in account.get('Monthly_Details', []):
                account_activity_data.append({
                    'Customer_ID': customer['Customer_ID'],
                    'Account_ID': account['Account_ID'],
                    **monthly_detail
                })

    account_activity_df = pd.DataFrame(account_activity_data)
    account_activity_df.to_csv('account_activity_data.csv', index=False)

    end_time = time.time()
    elapsed_time = end_time - start_time

    logging.info(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Main - Execution completed in {elapsed_time:.2f} seconds.")
    logging.info(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Main - Total memory used: {memory_usage()[0]} MB")

    # Sample results in Jupyter Notebook
    print("Sample results:")
    print(customers_df.head())
    print(accounts_df.head())
    print(account_activity_df.head())
    #logs are to be printed at the end



#Following code is to calculate and display results of Functional Testing

# Customer Data
#Cohort information

# Load the customers data from the CSV file
file_path = 'customers_data.csv'
data = pd.read_csv(file_path)

# 1. Number of customers
num_customers = len(data)

# 2. Minimum and Maximum Customer ID
min_customer_id = data['Customer_ID'].min()
max_customer_id = data['Customer_ID'].max()

# 3. Number of unique Customer IDs
unique_customer_ids = data['Customer_ID'].nunique()

# 4. Descriptive statistics for the age of customers
age_stats = data['Age'].describe(percentiles=[.25, .5, .75])

# 5. Frequency table for Gender
gender_freq_table = data['Gender'].value_counts()
gender_percent_freq_table = data['Gender'].value_counts(normalize=True)
gender_cumulative_freq_table = gender_freq_table.cumsum()
gender_cumulative_percent_freq_table = gender_percent_freq_table.cumsum()

# 6. Frequency table for Marital Status
marital_status_freq_table = data['Marital_Status'].value_counts()
marital_status_percent_freq_table = data['Marital_Status'].value_counts(normalize=True)
marital_status_cumulative_freq_table = marital_status_freq_table.cumsum()
marital_status_cumulative_percent_freq_table = marital_status_percent_freq_table.cumsum()

# 7. Percent frequency of Marital Status for specific age categories
age_bins = [20, 30, 60, 80]
data['Age_Category'] = pd.cut(data['Age'], bins=age_bins, labels=['20-30', '30-60', '60-80'])

# 8. Frequency table for Number of Children
children_freq_table = data['Number_of_Children'].value_counts()
children_percent_freq_table = data['Number_of_Children'].value_counts(normalize=True)
children_cumulative_freq_table = children_freq_table.cumsum()
children_cumulative_percent_freq_table = children_percent_freq_table.cumsum()

# 9. Percent frequency of Number of Children for specific age categories
age_bins_children = [20, 40, 80]
data['Age_Category_Children'] = pd.cut(data['Age'], bins=age_bins_children, labels=['20-40', '40-80'])
children_age_category_percent_freq = pd.crosstab(data['Age_Category_Children'], data['Number_of_Children'],
                                                 normalize='index')
children_age_category_cross_tab = pd.crosstab(data['Age_Category_Children'], data['Number_of_Children'],
                                              normalize='index')

# 10. Frequency table for Education Level
education_freq_table = data['Education_Level'].value_counts()
education_percent_freq_table = data['Education_Level'].value_counts(normalize=True)
education_cumulative_freq_table = education_freq_table.cumsum()
education_cumulative_percent_freq_table = education_percent_freq_table.cumsum()

# 11. Percent frequency of Education Level for specific age categories
age_bins_education = [20, 25, 35, 80]
data['Age_Category_Education'] = pd.cut(data['Age'], bins=age_bins_education, labels=['20-25', '25-35', '35-80'])
education_age_category_percent_freq = pd.crosstab(data['Age_Category_Education'], data['Education_Level'],
                                                  normalize='index')
education_age_category_cross_tab = pd.crosstab(data['Age_Category_Education'], data['Education_Level'],
                                               normalize='index')

# 12. Descriptive statistics for Annual Income
annual_income_stats = data['Annual_Income'].describe(percentiles=[.25, .5, .75])

# 13. Frequency table for Number of Accounts
num_accounts_freq_table = data['Number_of_Accounts'].value_counts()
num_accounts_percent_freq_table = data['Number_of_Accounts'].value_counts(normalize=True)
num_accounts_cumulative_freq_table = num_accounts_freq_table.cumsum()
num_accounts_cumulative_percent_freq_table = num_accounts_percent_freq_table.cumsum()

# 14. Descriptive statistics for Total Credit Line
total_credit_line_stats = data['Total_Credit_Line'].describe(percentiles=[.25, .5, .75])
total_credit_line_stats = total_credit_line_stats.to_frame().transpose()

# Create DataFrames for results
results = pd.DataFrame({
    'Number of customers': [num_customers],
    'Minimum Customer ID': [min_customer_id],
    'Maximum Customer ID': [max_customer_id],
    'Number of unique Customer IDs': [unique_customer_ids],
})

results_age = age_stats.to_frame().transpose()
results_gender = pd.DataFrame({'Category': gender_freq_table.index,
                               'Frequency': gender_freq_table.values,
                               'Percent Frequency': gender_percent_freq_table.values * 100,
                               'Cumulative Frequency': gender_cumulative_freq_table.values,
                               'Cumulative Percent Frequency': gender_cumulative_percent_freq_table.values * 100
                               })

marital_status_table = pd.DataFrame({
    'Category': marital_status_freq_table.index,
    'Frequency': marital_status_freq_table.values,
    'Percent Frequency': marital_status_percent_freq_table.values * 100,
    'Cumulative Frequency': marital_status_cumulative_freq_table.values,
    'Cumulative Percent Frequency': marital_status_cumulative_percent_freq_table.values * 100
})
marital_status_age_category_percent_freq = pd.crosstab(data['Age_Category'], data['Marital_Status'], normalize='index')

children_table = pd.DataFrame({
    'Category': children_freq_table.index,
    'Frequency': children_freq_table.values,
    'Percent Frequency': children_percent_freq_table.values * 100,
    'Cumulative Frequency': children_cumulative_freq_table.values,
    'Cumulative Percent Frequency': children_cumulative_percent_freq_table.values * 100
})

education_table = pd.DataFrame({
    'Category': education_freq_table.index,
    'Frequency': education_freq_table.values,
    'Percent Frequency': education_percent_freq_table.values * 100,
    'Cumulative Frequency': education_cumulative_freq_table.values,
    'Cumulative Percent Frequency': education_cumulative_percent_freq_table.values * 100
})

accounts_table = pd.DataFrame({
    'Category': num_accounts_freq_table.index,
    'Frequency': num_accounts_percent_freq_table.values,
    'Percent Frequency': num_accounts_percent_freq_table.values * 100,
    'Cumulative Frequency': num_accounts_cumulative_freq_table.values,
    'Cumulative Percent Frequency': num_accounts_cumulative_percent_freq_table.values * 100
})

#Also printing them to console
print('\n1-3. Customer Results: min, max and unique customers:')
display(results)
print('\n4. Age Results')
display(results_age)
print('\n5. Gender')
display(results_gender)
print('\n6. Marital Status')
display(marital_status_table)
print('\n7. Marital Status for specific age:')
print(marital_status_age_category_percent_freq)
print('\n8. Frequency table for children')
print(children_table)
print('\n9. Frequency of Number of Children')
print(children_age_category_cross_tab)
print('\n10. Frequency of Education Level')
print(education_table)
print('\n11. Education level for ages')
print(education_age_category_cross_tab)
print('\n12. Annual Income calculations')
print(annual_income_stats)
print('\n13. Accounts Table')
print(accounts_table)
print('\n14. Credit line Results')
print(total_credit_line_stats)





# B. Accounts information
# Load the data from the CSV file
file_path = 'accounts_data.csv'
accounts_data = pd.read_csv(file_path)

# 1. Min and Max Date Opened
min_date_opened = accounts_data['Date_Opened'].min()
max_date_opened = accounts_data['Date_Opened'].max()

# 2. Min, P25, Median, P75, Max, Mean, and Standard Deviation for Age of the accounts (in years)
accounts_data['Date_Opened'] = pd.to_datetime(accounts_data['Date_Opened'])
accounts_data['Account_Age'] = (pd.to_datetime('2022-01-01') - accounts_data['Date_Opened']).dt.days // 365

age_stats = accounts_data['Account_Age'].describe(percentiles=[.25, .5, .75])

# 3. Frequency table for Account Age Flag
accounts_data['Account_Age_Flag'] = accounts_data['Account_Age'] >= 20
account_age_flag_freq_table = accounts_data['Account_Age_Flag'].value_counts()
account_age_flag_percent_freq_table = accounts_data['Account_Age_Flag'].value_counts(normalize=True)
account_age_flag_cumulative_freq_table = account_age_flag_freq_table.cumsum()
account_age_flag_cumulative_percent_freq_table = account_age_flag_percent_freq_table.cumsum()

# 4. Min and Max Account Number
min_account_number = accounts_data['Account_ID'].min()
max_account_number = accounts_data['Account_ID'].max()

# 5. Frequency table for last digit of the Account Number
accounts_data['Last_Digit_Account_Number'] = accounts_data['Account_ID'] % 10
last_digit_freq_table = accounts_data['Last_Digit_Account_Number'].value_counts()
last_digit_percent_freq_table = accounts_data['Last_Digit_Account_Number'].value_counts(normalize=True)
last_digit_cumulative_freq_table = last_digit_freq_table.cumsum()
last_digit_cumulative_percent_freq_table = last_digit_percent_freq_table.cumsum()

# 6. Min, P25, Median, P75, Max, Mean, and Standard Deviation for Account Credit Line
credit_line_stats = accounts_data['Account_Credit_Line'].describe(percentiles=[.25, .5, .75])

# 7. Frequency table for Account Credit Line Flag
accounts_data['Account_Credit_Line_Flag'] = accounts_data['Account_Credit_Line'] == accounts_data[
    'Account_Credit_Line'].sum()
credit_line_flag_freq_table = accounts_data['Account_Credit_Line_Flag'].value_counts()
credit_line_flag_percent_freq_table = accounts_data['Account_Credit_Line_Flag'].value_counts(normalize=True)
credit_line_flag_cumulative_freq_table = credit_line_flag_freq_table.cumsum()
credit_line_flag_cumulative_percent_freq_table = credit_line_flag_percent_freq_table.cumsum()

# 8. Min, P25, Median, P75, Max, Mean, and Standard Deviation for Annual Fee
annual_fee_stats = accounts_data['Annual_Fee'].describe(percentiles=[.25, .5, .75])

# 9. Frequency table for Annual Fee Flag
accounts_data['Annual_Fee_Flag'] = accounts_data['Annual_Fee'] == accounts_data['Account_Credit_Line'] * 0.01
annual_fee_flag_freq_table = accounts_data['Annual_Fee_Flag'].value_counts()
annual_fee_flag_percent_freq_table = accounts_data['Annual_Fee_Flag'].value_counts(normalize=True)
annual_fee_flag_cumulative_freq_table = annual_fee_flag_freq_table.cumsum()
annual_fee_flag_cumulative_percent_freq_table = annual_fee_flag_percent_freq_table.cumsum()

# 10. Min, P25, Median, P75, Max, Mean, and Standard Deviation for Annual Interest Rate
interest_rate_stats = accounts_data['Annual_Interest_Rate'].describe(percentiles=[.25, .5, .75])


# Display the results
print("\n1. Min and Max Date Opened:")
print(f"Min Date Opened: {min_date_opened}")
print(f"Max Date Opened: {max_date_opened}")

print("\n2. Min, P25, Median, P75, Max, Mean, and Standard Deviation for Age of the accounts (in years):")
print(age_stats)

print("\n3. Frequency table for Account Age Flag:")
print(account_age_flag_freq_table)
print(account_age_flag_percent_freq_table)
print(account_age_flag_cumulative_freq_table)
print(account_age_flag_cumulative_percent_freq_table)

print("\n4. Min and Max Account Number:")
print(f"Min Account Number: {min_account_number}")
print(f"Max Account Number: {max_account_number}")

print("\n5. Frequency table for last digit of the Account Number:")
print(last_digit_freq_table)
print(last_digit_percent_freq_table)
print(last_digit_cumulative_freq_table)
print(last_digit_cumulative_percent_freq_table)

print("\n6. Min, P25, Median, P75, Max, Mean, and Standard Deviation for Account Credit Line:")
print(credit_line_stats)

print("\n7. Frequency table for Account Credit Line Flag:")
print(credit_line_flag_freq_table)
print(credit_line_flag_percent_freq_table)
print(credit_line_flag_cumulative_freq_table)
print(credit_line_flag_cumulative_percent_freq_table)

print("\n8. Min, P25, Median, P75, Max, Mean, and Standard Deviation for Annual Fee:")
print(annual_fee_stats)

print("\n9. Frequency table for Annual Fee Flag:")
print(annual_fee_flag_freq_table)
print(annual_fee_flag_percent_freq_table)
print(annual_fee_flag_cumulative_freq_table)
print(annual_fee_flag_cumulative_percent_freq_table)

print("\n10. Min, P25, Median, P75, Max, Mean, and Standard Deviation for Annual Interest Rate:")
print(interest_rate_stats)



# C. Account Activity Information
# Load the data from the CSV file
file_path = 'account_activity_data.csv'
activity_data = pd.read_csv(file_path)

# 1. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Transactions
transaction_stats = activity_data['Total_Transactions_of_month'].describe(percentiles=[.25, .5, .75])

# 2. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Purchases
purchase_stats = activity_data['Purchase_Transactions_of_month'].describe(percentiles=[.25, .5, .75])

# 3. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Cash Advances
cash_advance_stats = activity_data['Cash_advance_Transactions_of_month'].describe(percentiles=[.25, .5, .75])

# 4. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Purchase Amounts
purchase_amount_stats = activity_data['Total_Purchases'].describe(percentiles=[.25, .5, .75])

# 5. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Cash Advance Amounts
cash_advance_amount_stats = activity_data['Total_Cash_Advances'].describe(percentiles=[.25, .5, .75])

# 6. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Payments Amounts
payment_amount_stats = activity_data['Total_Payments'].describe(percentiles=[.25, .5, .75])

# 7. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Closing Balances
closing_balance_stats = activity_data['Closing_Balance'].describe(percentiles=[.25, .5, .75])

# 8. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Minimum Amounts Due
min_amount_due_stats = activity_data['Minimum_Amount_Due'].describe(percentiles=[.25, .5, .75])

# Convert the string representation of arrays to actual arrays
activity_data['Total_Individual_Payments_of_month'] = activity_data['Total_Individual_Payments_of_month'].apply(eval)
activity_data['Total_Individual_Purchases_of_month'] = activity_data['Total_Individual_purchases_of_month'].apply(eval)
activity_data['Total_Individual_Cash_Advances_of_month'] = activity_data[
    'Total_Individual_Cash_Advances_of_month'].apply(eval)


def calculate_stats(array):
    if len(array) == 0:
        return [np.nan] * 7  # Return NaN values for empty arrays

    stats = np.percentile(array, [0, 25, 50, 75, 100])  # Percentiles: 0, 25, 50 (median), 75, 100
    stats = np.append(stats, [np.mean(array), np.std(array)])  # Mean and Standard Deviation
    return stats


# Calculate statistics for each array column
payment_stats = activity_data['Total_Individual_Payments_of_month'].apply(calculate_stats).apply(pd.Series)
purchase_stats = activity_data['Total_Individual_Purchases_of_month'].apply(calculate_stats).apply(pd.Series)
cash_advance_stats = activity_data['Total_Individual_Cash_Advances_of_month'].apply(calculate_stats).apply(pd.Series)

# Rename the columns for better clarity
payment_stats.columns = ['Min_Payment', '25th_Percentile_Payment', 'Median_Payment', '75th_Percentile_Payment',
                         'Max_Payment', 'Mean_Payment', 'Std_Dev_Payment']
purchase_stats.columns = ['Min_Purchase', '25th_Percentile_Purchase', 'Median_Purchase', '75th_Percentile_Purchase',
                          'Max_Purchase', 'Mean_Purchase', 'Std_Dev_Purchase']
cash_advance_stats.columns = ['Min_Cash_Advance', '25th_Percentile_Cash_Advance', 'Median_Cash_Advance',
                              '75th_Percentile_Cash_Advance', 'Max_Cash_Advance', 'Mean_Cash_Advance',
                              'Std_Dev_Cash_Advance']

# 12. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Total Interests of the month
total_interest_stats = activity_data['Total_Interest_Charged'].describe(percentiles=[.25, .5, .75])

# 13. Frequency table for Delinquency Counter
delinquency_counter_freq_table = activity_data['Delinquency_Count'].value_counts()
delinquency_counter_percent_freq_table = activity_data['Delinquency_Count'].value_counts(normalize=True)
delinquency_counter_cumulative_freq_table = delinquency_counter_freq_table.cumsum()
delinquency_counter_cumulative_percent_freq_table = delinquency_counter_percent_freq_table.cumsum()

# 14. Frequency table for Annual Active Flag
activity_data['Annual_Active_Flag'] = activity_data['Delinquency_Count'] < 3
annual_active_flag_freq_table = activity_data['Annual_Active_Flag'].value_counts()
annual_active_flag_percent_freq_table = activity_data['Annual_Active_Flag'].value_counts(normalize=True)
annual_active_flag_cumulative_freq_table = annual_active_flag_freq_table.cumsum()
annual_active_flag_cumulative_percent_freq_table = annual_active_flag_percent_freq_table.cumsum()

# Display the results of Min, P25, Median, P75, Max, Mean, and Standard Deviations of calculated data
print("\n1. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Transactions:")
print(transaction_stats)

print("\n2. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Purchases:")
print(purchase_stats)

print("\n3. Min, P25, Median, P75, Max, Mean, and Standard Deviation for the Number of Cash Advances:")
print(cash_advance_stats)

print("\n4. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Purchase Amounts:")
print(purchase_amount_stats.to_frame().transpose())

print("\n5. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Cash Advance Amounts:")
print(cash_advance_amount_stats)

print("\n6. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Payments Amounts:")
print(payment_amount_stats.to_frame().transpose())

print("\n7. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Closing Balances:")
print(closing_balance_stats.to_frame().transpose())

print("\n8. Min, P25, Median, P75, Max, Mean, and Standard Deviation for all the Minimum Amounts Due:")
print(min_amount_due_stats)

print(
    "\n9. Min, 25percent, Median, 75percent, Max, Mean, and Standard Deviation for all the Payment Amounts of the month:")
print(payment_stats.head())

print(
    "\n10.Min, 25percent, Median, 75percent, Max, Mean, and Standard Deviation for all the Total Purchase Amounts of the month:")
print(purchase_stats.head())

print("\n11.Min, 25percent, Median, 75percent, Max, Mean, and Standard Deviation for all the Total Cash Advance Amounts of the month:")
print(cash_advance_stats.head())

print("\n13. Frequency table for Delinquency Counter:")
print(delinquency_counter_freq_table)
print(delinquency_counter_percent_freq_table)
print(delinquency_counter_cumulative_freq_table)
print(delinquency_counter_cumulative_percent_freq_table)

print("\n14. Frequency table for Annual Active Flag:")
print(annual_active_flag_freq_table)
print(annual_active_flag_percent_freq_table)
print(annual_active_flag_cumulative_freq_table)
print(annual_active_flag_cumulative_percent_freq_table)

#Printing the logs into the console 
#sometimes the data rate exceeds and logs are not printed to console but are stored in log file
print("Logs:")
with open('app.log', 'r') as log_file:
    print(log_file.read())

Sample results:
   Customer_ID  Age  Gender Marital_Status  Number_of_Children  \
0      5061525   52    Male        Married                   3   
1      1826965   34    Male         Single                   2   
2      3147168   33    Male         Single                   0   
3      4980124   75  Female         Single                   1   
4      4868155   76    Male         Single                   2   

     Education_Level  Annual_Income  Number_of_Accounts  Total_Credit_Line  
0        High School        73632.0                   5      260312.772869  
1              Ph.D.       128544.0                   3      119482.191085  
2  Bachelor's Degree        86528.0                   1       12931.487789  
3    Master's Degree       124800.0                   2       67210.126783  
4        High School        83616.0                   3       69387.064042  
   Customer_ID  Account_ID Date_Opened  Account_Credit_Line   Annual_Fee  \
0      5061525    50615251  2009-08-16         12

,Number of customers,Minimum Customer ID,Maximum Customer ID,Number of unique Customer IDs
0,20000,1001695,9999711,19982



4. Age Results


,count,mean,std,min,25%,50%,75%,max
Age,20000.0,50.03035,17.604713,20.0,35.0,50.0,65.0,80.0



5. Gender


,Category,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
0,Male,10080,50.4,10080,50.4
1,Female,9920,49.6,20000,100.0



6. Marital Status


,Category,Frequency,Percent Frequency,Cumulative Frequency,Cumulative Percent Frequency
0,Married,11574,57.87,11574,57.87
1,Single,8426,42.13,20000,100.00



7. Marital Status for specific age:
Marital_Status   Married    Single
Age_Category                      
20-30           0.242481  0.757519
30-60           0.747542  0.252458
60-80           0.508694  0.491306

8. Frequency table for children
   Category  Frequency  Percent Frequency  Cumulative Frequency  \
0         1       6012             30.060                  6012   
1         2       5327             26.635                 11339   
2         0       4010             20.050                 15349   
3         3       3298             16.490                 18647   
4         4       1353              6.765                 20000   

   Cumulative Percent Frequency  
0                        30.060  
1                        56.695  
2                        76.745  
3                        93.235  
4                       100.000  

9. Frequency of Number of Children
Number_of_Children            0         1         2         3         4
Age_Category_Children                   

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

